In [1]:
import pandas as pd
from tqdm import tqdm
from pandarallel import pandarallel

In [2]:
df_main = pd.read_csv("modded_recount3_inc")
df_main

,chrom,intron_id,intron_start,intron_ends,strand,Duplicate_Count,RC3
0,chr1,0,12227,12612,+,2,['GTExv2=1122:3199;TCGAv2=583:1460;SRAv3h=9374...
1,chr1,1,12721,13220,+,1,['GTExv2=1791:3198;TCGAv2=783:1104;SRAv3h=1404...
2,chr1,0,12057,12178,+,1,"[None, -1]"
3,chr1,2,12697,12974,+,1,['GTExv2=1:1;TCGAv2=1:1;SRAv3h=15:19;GT:AG' 21]
4,chr1,3,13052,13220,+,1,['GTExv2=22:24;TCGAv2=17:18;SRAv3h=433:484;GC:...
...,...,...,...,...,...,...,...
517017,chrY,0,15955,57212183,-,1,"[None, -1]"
517018,chrY,1,57213125,57213203,-,1,"[None, -1]"
517019,chrY,2,57213357,57213525,-,1,"[None, -1]"
517020,chrY,3,57213602,57213879,-,1,"[None, -1]"


In [3]:
def match_repeat_features(row):
    # print(row)
    # print(row.chrom)
    # try:
        # score = df_recount3_test.loc[(row.strand, row.intron_start, row.intron_ends), ["Features", "Score"]].values
    # !echo 'tabix ftp-data/recount3.bed.gz {row.chrom}:{row.intron_start}-{row.intron_ends}'
    # !echo tabix ftp-data/repeat_features.bed.gz {row.chrom}:{row.intron_start}-{row.intron_ends}
    matched_rows = !tabix ftp-data/repeat_features.bed.gz {row.chrom}:{row.intron_start}-{row.intron_ends} | cut -f4 | sort -u
    # | awk -F'\t' '{print $4}' | sort -u
    #|    awk -v start={row.intron_start} -v stop={row.intron_ends} '$2 == start && $3 == stop'
    # score = !tabix ftp-data/recount3.bed.gz chr1:12227-12612 | awk -v start=12227 -v stop=12612 '$2 == start && $3 == stop'

        #"Chrom", "Score", "GTExv2", "TCGAv2", "SRAv3h", "Splice_site"
    # except:
    #     print("ERROR")
    #     score = [None, -1]
    #     score = "Unmatched"
    # print(df_matched)
    # print(matched_rows)
    # print(type(score))
    return list(matched_rows)

In [4]:
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
%%time
df_draft = df_main.copy().head(50)
df_draft["Repeat_overlap"] = df_draft.parallel_apply(match_repeat_features, axis=1)
df_draft

In [5]:
%%time
df_main["Repeat_overlaps"] = df_main.parallel_apply(match_repeat_features, axis=1)
df_main

CPU times: user 436 ms, sys: 84.7 ms, total: 521 ms
Wall time: 5min 50s


,chrom,intron_id,intron_start,intron_ends,strand,Duplicate_Count,RC3,Repeat_overlaps
0,chr1,0,12227,12612,+,2,['GTExv2=1122:3199;TCGAv2=583:1460;SRAv3h=9374...,[]
1,chr1,1,12721,13220,+,1,['GTExv2=1791:3198;TCGAv2=783:1104;SRAv3h=1404...,[]
2,chr1,0,12057,12178,+,1,"[None, -1]",[]
3,chr1,2,12697,12974,+,1,['GTExv2=1:1;TCGAv2=1:1;SRAv3h=15:19;GT:AG' 21],[]
4,chr1,3,13052,13220,+,1,['GTExv2=22:24;TCGAv2=17:18;SRAv3h=433:484;GC:...,[]
...,...,...,...,...,...,...,...,...
517017,chrY,0,15955,57212183,-,1,"[None, -1]","[Centromere, Dust, Low complexity regions, LTR..."
517018,chrY,1,57213125,57213203,-,1,"[None, -1]",[]
517019,chrY,2,57213357,57213525,-,1,"[None, -1]",[]
517020,chrY,3,57213602,57213879,-,1,"[None, -1]",[]


In [ ]:
df_main

In [6]:
df_main.to_csv("modded_repeat_features", index=False)